In [1]:
import nd2
import torch
import numpy as np
from dask import delayed

In [2]:
from dask.distributed import Client
client = Client(n_workers=4)

In [3]:
def load_data_delayed(file: str = "../data/A1_s3001.nd2", num_frames: int = 500):
    data = nd2.imread(file, xarray=False, dask=True)
    data = data[:num_frames,:,:]
    return data

def load_data(file: str = "../data/A1_s3001.nd2", num_frames: int = 500):
    data = nd2.imread(file, xarray=False, dask=False)
    data = data[:num_frames,:,:]
    return data

def numpy_fft2(data: np.ndarray):
    data2 = np.fft.fft2(data)
    return np.fft.ifft2(data2)

def torch_fft2(data: np.ndarray):
    data2 = torch.fft.fft2(data)    
    return torch.fft.ifft2(data2)

In [4]:
data = load_data_delayed()
data

dask.array<getitem, shape=(500, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>

In [7]:
%%time
for image in data:
    tensor = delayed(torch.from_numpy)(np.asarray(image, dtype="float32"))
    torch_out = delayed(torch_fft2)(tensor)
    out_torch = torch_out.compute()


CPU times: total: 32.3 s
Wall time: 1min 47s


In [6]:
%%time
result = numpy_fft2(data)
out_np = result.compute()

CPU times: total: 21.5 s
Wall time: 58.2 s


In [ ]:
data = load_data()

In [ ]:
%%time
result = np.empty((512, 512))

for image in data:
    out = numpy_fft2(image)
    # result = np.append(result, out)

In [ ]:
result